In [1]:
import numpy as np
import pandas as pd
import math
import re

In [2]:
df_preds = pd.read_csv('./../Responses/DF_of_GPT5_Response_v1.csv')

print(df_preds.shape)
df_preds.head()

(198, 4)


,input_column,matched_KF_column_1,matched_KF_column_2,matched_KF_column_3
0,Country Code,Country,Currency,Employee Work Location / Zip/Postal Code
1,Case Number,Employee ID,Korn Ferry Reference Job Code,Legal Entity / Business Unit
2,Reports to Case Number,Manager Employee ID,Employee ID,Legal Entity / Business Unit
3,Area/City Code,Employee Work Location / Zip/Postal Code,Country,Legal Entity / Business Unit
4,Postal/ZIP Code,Employee Work Location / Zip/Postal Code,Country,Legal Entity / Business Unit


In [ ]:
df_gt = pd.read_excel('./../Files/Ground_Truth/mappings_simple_All Countries - 2025 Submission_WTW_test_ORGDC_BA_General_en_IDC_20250928_150318_audit 1 Jakub__summary.xlsx')

print(df_gt.shape)
df_gt.head()

(170, 7)


,Input Header,Proposed Template Column,GroundTruth Template Column,Is_Correct,Stage,Confidence_Level,Decision
0,Gender,GENDER,GENDER,True,1,auto_accept,Mapped
1,Function Code,FUNCTION CODE (NOT REQUIRED IF REF. JOB CODE ...,NaN,False,1,auto_accept,Mapped
2,Housing Allowance,HOUSING ALLOWANCE (ANNUAL FULL-TIME EQUIVALENT),HOUSING ALLOWANCE (ANNUAL FULL-TIME EQUIVALENT),True,1,auto_accept,Mapped
3,Meal Allowance,MEAL ALLOWANCE (ANNUAL FULL-TIME EQUIVALENT),MEAL ALLOWANCE (ANNUAL FULL-TIME EQUIVALENT),True,1,auto_accept,Mapped
4,Telecommunication Allowance,TELECOMMUNICATION ALLOWANCE (ANNUAL FULL-TIME ...,TELECOMMUNICATION ALLOWANCE (ANNUAL FULL-TIME ...,True,1,auto_accept,Mapped


In [8]:
# wherever Input Headers match, get the corresponding GT Template IDs
df_merged = pd.merge(df_preds, df_gt[['Input Header', 'GroundTruth Template Column']], left_on='input_column', right_on='Input Header', how='left')

print(df_merged.shape)
df_merged.head()

(198, 6)


,input_column,matched_KF_column_1,matched_KF_column_2,matched_KF_column_3,Input Header,GroundTruth Template Column
0,Country Code,Country,Currency,Employee Work Location / Zip/Postal Code,NaN,NaN
1,Case Number,Employee ID,Korn Ferry Reference Job Code,Legal Entity / Business Unit,NaN,NaN
2,Reports to Case Number,Manager Employee ID,Employee ID,Legal Entity / Business Unit,Reports to Case Number,NaN
3,Area/City Code,Employee Work Location / Zip/Postal Code,Country,Legal Entity / Business Unit,Area/City Code,EMPLOYEE WORK LOCATION / ZIP/POSTAL CODE
4,Postal/ZIP Code,Employee Work Location / Zip/Postal Code,Country,Legal Entity / Business Unit,Postal/ZIP Code,EMPLOYEE WORK LOCATION / ZIP/POSTAL CODE


In [9]:
df_merged_notNaN = df_merged[~df_merged['GroundTruth Template Column'].isna()]
print(df_merged_notNaN.shape)
df_merged_notNaN.head()

(72, 6)


,input_column,matched_KF_column_1,matched_KF_column_2,matched_KF_column_3,Input Header,GroundTruth Template Column
3,Area/City Code,Employee Work Location / Zip/Postal Code,Country,Legal Entity / Business Unit,Area/City Code,EMPLOYEE WORK LOCATION / ZIP/POSTAL CODE
4,Postal/ZIP Code,Employee Work Location / Zip/Postal Code,Country,Legal Entity / Business Unit,Postal/ZIP Code,EMPLOYEE WORK LOCATION / ZIP/POSTAL CODE
5,Unique Incumbent Identifier,Employee ID,Manager Employee ID,Korn Ferry Reference Job Code,Unique Incumbent Identifier,EMPLOYEE ID
6,Department Name,Department,Legal Entity / Business Unit,Function Code (not required if Ref. Job Code ...,Department Name,DEPARTMENT
7,Internal Position Title,Employee Job Title,Korn Ferry Reference Job Code,Korn Ferry Reference Level (not required if ...,Internal Position Title,EMPLOYEE JOB TITLE


In [10]:
# calculate col1 acc, col2 acc, and col3 acc
# after making strings lowercase and stripping whitespace
df_merged_notNaN['matched_KF_column_1'] = df_merged_notNaN['matched_KF_column_1'].str.lower().str.strip()
df_merged_notNaN['matched_KF_column_2'] = df_merged_notNaN['matched_KF_column_2'].str.lower().str.strip()
df_merged_notNaN['matched_KF_column_3'] = df_merged_notNaN['matched_KF_column_3'].str.lower().str.strip()
df_merged_notNaN['GroundTruth Template Column'] = df_merged_notNaN['GroundTruth Template Column'].str.lower().str.strip()

df_merged_notNaN.head()

/var/folders/8b/nppjnl6j1xq3tmzf0rzzd8sr0000gp/T/ipykernel_38382/2065101339.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_merged_notNaN['matched_KF_column_1'] = df_merged_notNaN['matched_KF_column_1'].str.lower().str.strip()
/var/folders/8b/nppjnl6j1xq3tmzf0rzzd8sr0000gp/T/ipykernel_38382/2065101339.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_merged_notNaN['matched_KF_column_2'] = df_merged_notNaN['matched_KF_column_2'].str.lower().str.strip()
/var/folders/8b/nppjnl6j1xq3tmzf0rzzd8sr0000

,input_column,matched_KF_column_1,matched_KF_column_2,matched_KF_column_3,Input Header,GroundTruth Template Column
3,Area/City Code,employee work location / zip/postal code,country,legal entity / business unit,Area/City Code,employee work location / zip/postal code
4,Postal/ZIP Code,employee work location / zip/postal code,country,legal entity / business unit,Postal/ZIP Code,employee work location / zip/postal code
5,Unique Incumbent Identifier,employee id,manager employee id,korn ferry reference job code,Unique Incumbent Identifier,employee id
6,Department Name,department,legal entity / business unit,function code (not required if ref. job code ...,Department Name,department
7,Internal Position Title,employee job title,korn ferry reference job code,korn ferry reference level (not required if ...,Internal Position Title,employee job title


In [11]:
# remove '\n' from GroundTruth Template Column
df_merged_notNaN['GroundTruth Template Column'] = df_merged_notNaN['GroundTruth Template Column'].str.replace('\n', ' ')
df_merged_notNaN.head()

/var/folders/8b/nppjnl6j1xq3tmzf0rzzd8sr0000gp/T/ipykernel_38382/1942819303.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_merged_notNaN['GroundTruth Template Column'] = df_merged_notNaN['GroundTruth Template Column'].str.replace('\n', ' ')


,input_column,matched_KF_column_1,matched_KF_column_2,matched_KF_column_3,Input Header,GroundTruth Template Column
3,Area/City Code,employee work location / zip/postal code,country,legal entity / business unit,Area/City Code,employee work location / zip/postal code
4,Postal/ZIP Code,employee work location / zip/postal code,country,legal entity / business unit,Postal/ZIP Code,employee work location / zip/postal code
5,Unique Incumbent Identifier,employee id,manager employee id,korn ferry reference job code,Unique Incumbent Identifier,employee id
6,Department Name,department,legal entity / business unit,function code (not required if ref. job code ...,Department Name,department
7,Internal Position Title,employee job title,korn ferry reference job code,korn ferry reference level (not required if ...,Internal Position Title,employee job title


In [14]:
print(df_merged_notNaN.iloc[0]['matched_KF_column_1'])
print(df_merged_notNaN.iloc[0]['GroundTruth Template Column'])
print(df_merged_notNaN.iloc[0]['matched_KF_column_1'] == df_merged_notNaN.iloc[0]['GroundTruth Template Column'])

employee work location / zip/postal code
employee work location / zip/postal code
True


In [15]:
# calculate accuracy
correct_preds_1 = df_merged_notNaN[df_merged_notNaN['matched_KF_column_1'] == df_merged_notNaN['GroundTruth Template Column']]
correct_preds_2 = df_merged_notNaN[df_merged_notNaN['matched_KF_column_2'] == df_merged_notNaN['GroundTruth Template Column']]
correct_preds_3 = df_merged_notNaN[df_merged_notNaN['matched_KF_column_3'] == df_merged_notNaN['GroundTruth Template Column']]

acc_top1 = len(correct_preds_1) / len(df_merged_notNaN)
acc_top2 = (len(correct_preds_1) + len(correct_preds_2)) / len(df_merged_notNaN)
acc_top3 = (len(correct_preds_1) + len(correct_preds_2) + len(correct_preds_3)) / len(df_merged_notNaN)

print(f'Top-1 Accuracy: {acc_top1:.4f}')
print(f'Top-2 Accuracy: {acc_top2:.4f}')
print(f'Top-3 Accuracy: {acc_top3:.4f}')

Top-1 Accuracy: 0.8056
Top-2 Accuracy: 0.9167
Top-3 Accuracy: 0.9444
